# Binary classification risk control - Theoretical tests to validate implementation

# 1. The case of a random classifier

## 1.1. Protocol description
We test the theoretical guarantees of risk control in binary classification by using a random classifier and synthetic data. The aim is to evaluate the effectiveness of the BinaryClassificationController in maintaining a predefined risk level under different conditions.

Each test case looks at a combination of parameters, for which we repeat the experiment `n_repeat` times. The model is the same for all experiments (basically a random classifier), but the data is different each time.

Each experiment consists of the following:
 - We calibrate a BinaryClassificationController. It gives us the list of lambda values that control the risk according to LTT.
 - Because we know that the model is random, we know the theoretical risk associated with each lambda value. So we are able to check if the lambda values given by LTT actually control the risk. If not, we count 1 "error". Note that *each* lambda value should control the risk, not just one of them.

After n_repeat experiments, we compute the proportion of errors, that should be less than delta (1 - confidence_level).

## 1.2. Results
The risk is controlled in all the test cases. Overall, LTT seems very conservative (to achieve a high percentage of errors, we need to lower the confidence level significantly (0.01) and use only one threshold to avoid the Bonferroni effect). But this is likely due to the model being random, and thus having a lot of variance. It would be interesting to see how this evolves with a better model.

In [26]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from sklearn.datasets import make_classification
from sklearn.dummy import check_random_state
from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from itertools import product
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")

from mapie.risk_control import BinaryClassificationController, precision, accuracy, recall

In [28]:
# Using sklearn.dummy.DummyClassifier would be cleaner
class RandomClassifier:
    def __init__(self, seed=42, threshold=0.5):
        self.seed = seed
        self.threshold = threshold

    def _get_prob(self, x):
        local_seed = hash((x, self.seed)) % (2**32)
        rng = np.random.RandomState(local_seed)
        return np.round(rng.rand(), 2)

    def predict_proba(self, X):
        probs = np.array([self._get_prob(x) for x in X])
        return np.vstack([1 - probs, probs]).T

    def predict(self, X):
        probs = self.predict_proba(X)[:, 1]
        return (probs >= self.threshold).astype(int)


def precision_random_classifier(threshold: float) -> float:
    """
    Theoretical precision of RandomClassifier on a balanced dataset.
    - RandomClassifier assigns labels randomly with a uniform probability, independent of the true label.
    - Therefore, precision is always 0.5 regardless of the threshold.
    """
    return 0.5


def accuracy_random_classifier(threshold: float) -> float:
    """
    Theoretical accuracy of RandomClassifier on a balanced dataset.
    - RandomClassifier assigns labels randomly with a uniform probability, independent of the true label.
    - Therefore, accuracy is always 0.5 regardless of the threshold.
    """
    return 0.5


def recall_random_classifier(threshold: float) -> float:
    """
    Theoretical recall (sensitivity) of RandomClassifier on a balanced dataset.
    - RandomClassifier assigns labels randomly with a uniform probability, independent of the true label.
    - Therefore, recall = 1 - threshold.
    """
    return 1.0 - threshold

In [ ]:
def run_one_experiment_with_random_classifier(clf_class, risk_dict, predict_params, target_level, confidence_level, N, n_repeats):
    """
    Runs the experiment for one combination of risk, predict_params, target_level, confidence_level.
    Returns a DataFrame with one row per repeat.
    """
    clf = clf_class()
    records = []

    for repeat_id in range(n_repeats):
        X_calibrate, y_calibrate = make_classification(
            n_samples=N,
            n_features=1,
            n_informative=1,
            n_redundant=0,
            n_repeated=0,
            n_classes=2,
            n_clusters_per_class=1,
            weights=[0.5, 0.5],
            flip_y=0,
            random_state=None
        )
        X_calibrate = X_calibrate.squeeze()

        controller = BinaryClassificationController(
            predict_function=clf.predict_proba,
            risk=risk_dict["risk"],
            target_level=target_level,
            confidence_level=confidence_level,
            list_predict_params=predict_params
        )
        controller.calibrate(X_calibrate, y_calibrate)
        valid_parameters = controller.valid_predict_params

        error_indicator = 0

        if len(valid_parameters) == 0:
            error_indicator = 1
        else:
            for lambda_ in valid_parameters:
                if risk_dict["risk"] == precision:
                    theoretical_metric = precision_random_classifier(lambda_)
                elif risk_dict["risk"] == recall:
                    theoretical_metric = recall_random_classifier(lambda_)
                elif risk_dict["risk"] == accuracy:
                    theoretical_metric = accuracy_random_classifier(lambda_)

                if risk_dict["risk"].higher_is_better:
                    if theoretical_metric <= target_level:
                        error_indicator = 1
                        break
                else:
                    if theoretical_metric > target_level:
                        error_indicator = 1
                        break

        records.append({
            "risk_name": risk_dict["name"],
            "predict_param": predict_params,
            "target_level": target_level,
            "confidence_level": confidence_level,
            "repeat_id": repeat_id,
            "error_indicator": error_indicator,
            "valid_param": valid_parameters,
            "nb_valid_param": len(valid_parameters),
        })

    return pd.DataFrame(records)

In [71]:
def analyze_results(df_results):
    summary = []
    grouped = df_results.groupby(
        ["risk_name", "target_level", "confidence_level"]
    )

    for (risk_name, target_level, confidence_level), group in grouped:
        proportion_not_controlled = group["error_indicator"].mean()
        nb_predict_parameters = len(group.iloc[0]["predict_param"])
        mean_nb_valid_thresholds = group["nb_valid_param"].mean()

        delta = 1 - confidence_level
        valid_experiment = proportion_not_controlled <= delta

        summary.append({
            "risk_name": risk_name,
            "target_level": target_level,
            "confidence_level": confidence_level,
            "nb_predict_parameters": nb_predict_parameters,
            "proportion_not_controlled": proportion_not_controlled,
            "delta": delta,
            "mean_nb_valid_thresholds": mean_nb_valid_thresholds,
            "valid_experiment": valid_experiment
        })

    df_summary = pd.DataFrame(summary)

    return df_summary

In [ ]:
N = 100  # size of the calibration set
risk = [
    {"name": "precision", "risk": precision},
    {"name": "recall", "risk": recall},
    {"name": "accuracy", "risk": accuracy},
]
target_level = [0.1, 0.9]
confidence_level = [0.8, 0.2]
n_repeats = 100

print("#------------------Random classifier : the case of multiple parameters------------------#")
predict_params = [np.linspace(0, 0.99, 100)]
all_results = []

for combination in product(risk, predict_params, target_level, confidence_level):
    risk_dict, predict_param_set, t_level, c_level = combination

    df_one = run_one_experiment_with_random_classifier(
        clf_class=RandomClassifier,
        risk_dict=risk_dict,
        predict_params=predict_param_set,
        target_level=t_level,
        confidence_level=c_level,
        N=N,
        n_repeats=n_repeats
    )

    all_results.append(df_one)

df_results = pd.concat(all_results, ignore_index=True)
df_summary = analyze_results(df_results)
display(df_summary)

print("#------------------Random classifier : the case of single parameter------------------#")
all_results_single_param = []

for combination in product(risk, target_level, confidence_level):
    risk_dict, t_level, c_level = combination

    predict_param_set = np.array([np.random.choice(np.linspace(0, 0.9, 10))])

    df_one = run_one_experiment_with_random_classifier(
        clf_class=RandomClassifier,
        risk_dict=risk_dict,
        predict_params=predict_param_set,
        target_level=t_level,
        confidence_level=c_level,
        N=N,
        n_repeats=n_repeats
    )

    all_results_single_param.append(df_one)

df_results_single_param = pd.concat(all_results_single_param, ignore_index=True)
df_summary_single_param = analyze_results(df_results_single_param)
display(df_summary_single_param)


#------------------The case of multiple parameters------------------#


,risk_name,target_level,confidence_level,nb_predict_parameters,proportion_not_controlled,delta,mean_nb_valid_thresholds,valid_experiment
0,accuracy,0.1,0.2,100,0.00,0.8,100.00,True
1,accuracy,0.1,0.8,100,0.00,0.2,100.00,True
2,accuracy,0.9,0.2,100,0.00,0.8,0.00,True
3,accuracy,0.9,0.8,100,0.00,0.2,0.00,True
4,precision,0.1,0.2,100,0.00,0.8,92.23,True
5,precision,0.1,0.8,100,0.00,0.2,89.86,True
6,precision,0.9,0.2,100,0.00,0.8,0.00,True
7,precision,0.9,0.8,100,0.00,0.2,0.00,True
8,recall,0.1,0.2,100,0.01,0.8,77.65,True
9,recall,0.1,0.8,100,0.00,0.2,75.62,True


#------------------The case of single parameter------------------#


,risk_name,target_level,confidence_level,nb_predict_parameters,proportion_not_controlled,delta,mean_nb_valid_thresholds,valid_experiment
0,accuracy,0.1,0.2,1,0.0,0.8,1.00,True
1,accuracy,0.1,0.8,1,0.0,0.2,1.00,True
2,accuracy,0.9,0.2,1,0.0,0.8,0.00,True
3,accuracy,0.9,0.8,1,0.0,0.2,0.00,True
4,precision,0.1,0.2,1,0.0,0.8,1.00,True
5,precision,0.1,0.8,1,0.0,0.2,1.00,True
6,precision,0.9,0.2,1,0.0,0.8,0.00,True
7,precision,0.9,0.8,1,0.0,0.2,0.00,True
8,recall,0.1,0.2,1,0.0,0.8,1.00,True
9,recall,0.1,0.8,1,0.0,0.2,0.99,True


# 2. The case of a logistic classifier

## 2.1. Protocol description
We use the same protocol as described above, with the difference that we employ a logistic classifier and synthetic data generated from a logistic data generator.

Another difference is that, with this model, we do not know the theoretical risk associated with each value of lambda.
Therefore, we generate a sample of test data to estimate the risk for each lambda value.

## 2.2. Results
The risk is controlled in all test cases. Overall, LTT appears to be very conservative. It would be interesting to observe how this behavior evolves with a more accurate or complex model.

In [37]:
# Define a simple logistic classifier
class LogisticClassifier:
    """Deterministic sigmoid-based binary classifier."""

    def __init__(self, scale=2.0, threshold=0.5):
        self.scale = scale
        self.threshold = threshold
    
    def sigmoid(self, x):
        """Sigmoid function."""
        return 1 / (1 + np.exp(-self.scale * x))
    
    def _get_prob(self, x):
        """Probability of class 1 for input x."""
        return self.sigmoid(x)

    def predict_proba(self, X):
        """Return probabilities [p(y=0), p(y=1)] for each sample in X."""
        probs = np.array([self._get_prob(x) for x in X])
        return np.vstack([1 - probs, probs]).T

    def predict(self, X):
        """Return predicted class labels based on threshold."""
        probs = self.predict_proba(X)[:, 1]
        return (probs >= self.threshold).astype(int)

In [38]:
# Function to generate logistic data
def make_logistic_data(n_samples=200, scale=2.0, random_state=None):
    rng = check_random_state(random_state)
    X = rng.uniform(-3, 3, size=n_samples)
    probs = LogisticClassifier(scale=scale).sigmoid(X)
    y = rng.binomial(1, probs)
    return X, y

In [68]:
X, y = make_logistic_data(n_samples=300, scale=2.0, random_state=None)

x_curve = np.linspace(-3, 3, 400)
sigmoid_model = LogisticClassifier(scale=2.0)
y_curve = sigmoid_model.sigmoid(x_curve)

X_class_0 = X[y == 0]
X_class_1 = X[y == 1]
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=X_class_0,
    y=np.zeros_like(X_class_0),
    mode="markers",
    name='"negative" class',
    marker=dict(size=8),
))
fig.add_trace(go.Scatter(
    x=X_class_1,
    y=np.ones_like(X_class_1),
    mode="markers",
    name='"positive" class',
    marker=dict(size=8),
))

fig.add_trace(go.Scatter(
    x=x_curve,
    y=y_curve,
    mode="lines",
    name="Sigmoid",
    line=dict(width=3)
))
fig.update_layout(
    title="Logistic generated data and the sigmoid probability function",
    xaxis_title="X",
    yaxis_title="Probability and class labels",
    yaxis=dict(range=[-0.1, 1.1]),
    legend=dict(x=0.02, y=0.98),
    width=800,
    height=500
)
fig.show()


In [69]:
def run_one_experiment_with_logistic_classifier(clf_class, risk_dict, predict_params, target_level, confidence_level, N, n_repeats, scale=2.0):
    """
    Runs the experiment for one combination of using a LogisticClassifier.

    Returns a DataFrame with one row per repeat.
    """
    clf = clf_class(scale=scale, threshold=0.5)
    records = []

    for repeat_id in range(n_repeats):
        X_calibrate, y_calibrate = make_logistic_data(n_samples=N, scale=scale, random_state=None)

        controller = BinaryClassificationController(
            predict_function=clf.predict_proba,
            risk=risk_dict["risk"],
            target_level=target_level,
            confidence_level=confidence_level,
            list_predict_params=predict_params,
        )
        controller = controller.calibrate(X_calibrate, y_calibrate)
        valid_parameters = controller.valid_predict_params

        X_test, y_test = make_logistic_data(n_samples=N, scale=scale, random_state=None)
        probs = clf.predict_proba(X_test)[:, 1]

        error_indicator = 0

        if len(valid_parameters) == 0:
            error_indicator = 1
        else:
            for lambda_ in valid_parameters:
                y_pred = (probs >= lambda_).astype(int)

                if risk_dict["risk"] == precision:
                    empirical_metric = precision_score(y_test, y_pred, zero_division=0)
                elif risk_dict["risk"] == recall:
                    empirical_metric = recall_score(y_test, y_pred, zero_division=0)
                elif risk_dict["risk"] == accuracy:
                    empirical_metric = accuracy_score(y_test, y_pred)

                if risk_dict["risk"].higher_is_better:
                    if empirical_metric <= target_level:
                        error_indicator = 1
                        break
                else:
                    if empirical_metric > target_level:
                        error_indicator = 1
                        break

        records.append({
            "risk_name": risk_dict["name"],
            "predict_param": predict_params,
            "target_level": target_level,
            "confidence_level": confidence_level,
            "repeat_id": repeat_id,
            "error_indicator": error_indicator,
            "valid_param": valid_parameters,
            "nb_valid_param": len(valid_parameters),
        })

    return pd.DataFrame(records)

In [72]:
N = 100
risk = [
    {"name": "precision", "risk": precision},
    {"name": "recall", "risk": recall},
    {"name": "accuracy", "risk": accuracy},
]
target_level = [0.1, 0.9]
confidence_level = [0.8, 0.2]
n_repeats = 100
scale = 2.0

print("#------------------Logistic classifier : the case of multiple parameters------------------#")

predict_params = [np.linspace(0, 0.99, 100)]
all_results = []

for combination in product(risk, predict_params, target_level, confidence_level):
    risk_dict, predict_param_set, t_level, c_level = combination

    df_one = run_one_experiment_with_logistic_classifier(
        clf_class=LogisticClassifier,
        risk_dict=risk_dict,
        predict_params=predict_param_set,
        target_level=t_level,
        confidence_level=c_level,
        N=N,
        n_repeats=n_repeats,
        scale=scale
    )

    all_results.append(df_one)

df_results = pd.concat(all_results, ignore_index=True)
df_summary = analyze_results(df_results)
display(df_summary)

print("#------------------Logistic classifier : the case of single parameter------------------#")

all_results_single_param = []

for combination in product(risk, target_level, confidence_level):
    risk_dict, t_level, c_level = combination

    predict_param_set = np.array([np.random.choice(np.linspace(0, 0.9, 10))])

    df_one = run_one_experiment_with_logistic_classifier(
        clf_class=LogisticClassifier,
        risk_dict=risk_dict,
        predict_params=predict_param_set,
        target_level=t_level,
        confidence_level=c_level,
        N=N,
        n_repeats=n_repeats,
        scale=scale
    )

    all_results_single_param.append(df_one)

df_results_single_param = pd.concat(all_results_single_param, ignore_index=True)
df_summary_single_param = analyze_results(df_results_single_param)
display(df_summary_single_param)


#------------------Logistic classifier : the case of multiple parameters------------------#


,risk_name,target_level,confidence_level,nb_predict_parameters,proportion_not_controlled,delta,mean_nb_valid_thresholds,valid_experiment
0,accuracy,0.1,0.2,100,0.00,0.8,100.00,True
1,accuracy,0.1,0.8,100,0.00,0.2,100.00,True
2,accuracy,0.9,0.2,100,1.00,0.8,0.00,False
3,accuracy,0.9,0.8,100,1.00,0.2,0.00,False
4,precision,0.1,0.2,100,0.00,0.8,100.00,True
5,precision,0.1,0.8,100,0.00,0.2,100.00,True
6,precision,0.9,0.2,100,1.00,0.8,0.09,False
7,precision,0.9,0.8,100,1.00,0.2,0.00,False
8,recall,0.1,0.2,100,0.01,0.8,99.51,True
9,recall,0.1,0.8,100,0.00,0.2,99.35,True


#------------------Logistic classifier : the case of single parameter------------------#


,risk_name,target_level,confidence_level,nb_predict_parameters,proportion_not_controlled,delta,mean_nb_valid_thresholds,valid_experiment
0,accuracy,0.1,0.2,1,0.00,0.8,1.00,True
1,accuracy,0.1,0.8,1,0.00,0.2,1.00,True
2,accuracy,0.9,0.2,1,0.89,0.8,0.50,False
3,accuracy,0.9,0.8,1,1.00,0.2,0.00,False
4,precision,0.1,0.2,1,0.00,0.8,1.00,True
5,precision,0.1,0.8,1,0.00,0.2,1.00,True
6,precision,0.9,0.2,1,1.00,0.8,0.00,False
7,precision,0.9,0.8,1,0.89,0.2,0.15,False
8,recall,0.1,0.2,1,0.00,0.8,1.00,True
9,recall,0.1,0.8,1,0.00,0.2,1.00,True
